In [1]:
#!/usr/bin/env 

# User Inputs

In [2]:
#this is where all the files will be downloaded after the program is run
directory_path = '/home/user/Documents/python_webscraper/'

#Financial year 
year = 2020 

#Company name as it appears in the .tsv files
selectedcompany = 'WD 40 CO'

#when this is set to true the tsv documents pulled from from the SEC will be deleted after they are used.
purge_file_directory = False

# Modules

In [3]:
import edgar
import pandas as pd
import os

# Lists

In [4]:
tsv_file_names = []
#This is where we will store the balance sheets so that they do not get wiped out by the for loop
balance_sheets = []

# Definitions

In [5]:
#this function creates a list of file names that were created and that will be used to create the spreadsheets.
def tsv_name_formatter():
    for file in tsv_files:
        split_file = file.split('.')
        tsv_file_names.append(split_file[0])
        print('{} was created'.format(file))

In [6]:
#This will go through the documents and find the balance sheet.
def balance_sheet():
    for item in df:
        BS = (item[0].astype(str).str.contains('Retained') | item[0].astype(str).str.contains('Total Assets'))
        if BS.any():
            Balance_Sheet = item
            
            #Beginning stages of trimming down all the information that is irrelevant.
            Balance_Sheet = Balance_Sheet.iloc[10:57,[0,2,5]]

            #Creating a header and attaching it to the financial statement.
            header = Balance_Sheet.iloc[0]
            Balance_Sheet = Balance_Sheet[1:]
            Balance_Sheet.columns = header

            #Renaming the headers first cell 'item' since it usually begins as nan or na.
            Balance_Sheet.columns.values[0] = 'Item'
            Balance_Sheet = Balance_Sheet[Balance_Sheet['Item'].notna()]

            #The following lines complete the formatting of the financial statement so that it can be made into a proper dataframe.
            Balance_Sheet[Balance_Sheet.columns[1:]] = Balance_Sheet[Balance_Sheet.columns[1:]].astype(str)
            Balance_Sheet[Balance_Sheet.columns[1]] = Balance_Sheet[Balance_Sheet.columns[1]].map(lambda x: x.replace('-','0'))
            Balance_Sheet[Balance_Sheet.columns[2]] = Balance_Sheet[Balance_Sheet.columns[2]].map(lambda x: x.replace('-','0'))

            Balance_Sheet[Balance_Sheet.columns[1]] = Balance_Sheet[Balance_Sheet.columns[1]].map(lambda x: x.replace('(','-'))
            Balance_Sheet[Balance_Sheet.columns[2]] = Balance_Sheet[Balance_Sheet.columns[2]].map(lambda x: x.replace('(','-'))

            Balance_Sheet[Balance_Sheet.columns[1]] = Balance_Sheet[Balance_Sheet.columns[1]].map(lambda x: x.replace(')',''))
            Balance_Sheet[Balance_Sheet.columns[2]] = Balance_Sheet[Balance_Sheet.columns[2]].map(lambda x: x.replace(')',''))

            Balance_Sheet[Balance_Sheet.columns[1]] = Balance_Sheet[Balance_Sheet.columns[1]].map(lambda x: x.replace(',',''))
            Balance_Sheet[Balance_Sheet.columns[2]] = Balance_Sheet[Balance_Sheet.columns[2]].map(lambda x: x.replace(',',''))

            Balance_Sheet[Balance_Sheet.columns[1]] = Balance_Sheet[Balance_Sheet.columns[1]].map(lambda x: x.replace('nan','0'))
            Balance_Sheet[Balance_Sheet.columns[2]] = Balance_Sheet[Balance_Sheet.columns[2]].map(lambda x: x.replace('nan','0'))


            Balance_Sheet[Balance_Sheet.columns[1:]] = Balance_Sheet[Balance_Sheet.columns[1:]].astype(float)
            
            balance_sheets.append(Balance_Sheet)

In [11]:
def cash_flow_statment():
    for item in df:
        CFS = (item[0].astype(str).str.contains('CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS'))
        if CFS.any():
            Cash_Flow_Statement = item     
    
            Cash_Flow_Statement = Cash_Flow_Statement.iloc[6:48,[0,2,5]]
            
            header = Cash_Flow_Statement.iloc[0]
            Cash_Flow_Statement = Cash_Flow_Statement[1:]
            
            Cash_Flow_Statement.columns = header
            
            
            Cash_Flow_Statement.columns.values[0] = 'Item'
            Cash_Flow_Statement = Cash_Flow_Statement[Cash_Flow_Statement['Item'].notna()]
            
            Cash_Flow_Statement[Cash_Flow_Statement.columns[1:]] = Cash_Flow_Statement[Cash_Flow_Statement.columns[1:]].astype(str)
            Cash_Flow_Statement[Cash_Flow_Statement.columns[1]] = Cash_Flow_Statement[Cash_Flow_Statement.columns[1]].map(lambda x: x.replace('-','0'))
            Cash_Flow_Statement[Cash_Flow_Statement.columns[2]] = Cash_Flow_Statement[Cash_Flow_Statement.columns[2]].map(lambda x: x.replace('-','0'))
            
            Cash_Flow_Statement[Cash_Flow_Statement.columns[1]] = Cash_Flow_Statement[Cash_Flow_Statement.columns[1]].map(lambda x: x.replace('(','-'))
            Cash_Flow_Statement[Cash_Flow_Statement.columns[2]] = Cash_Flow_Statement[Cash_Flow_Statement.columns[2]].map(lambda x: x.replace('(','-'))
            
            Cash_Flow_Statement[Cash_Flow_Statement.columns[1]] = Cash_Flow_Statement[Cash_Flow_Statement.columns[1]].map(lambda x: x.replace(')',''))
            Cash_Flow_Statement[Cash_Flow_Statement.columns[2]] = Cash_Flow_Statement[Cash_Flow_Statement.columns[2]].map(lambda x: x.replace(')',''))
            
            Cash_Flow_Statement[Cash_Flow_Statement.columns[1]] = Cash_Flow_Statement[Cash_Flow_Statement.columns[1]].map(lambda x: x.replace(',',''))
            Cash_Flow_Statement[Cash_Flow_Statement.columns[2]] = Cash_Flow_Statement[Cash_Flow_Statement.columns[2]].map(lambda x: x.replace(',',''))
            
            Cash_Flow_Statement[Cash_Flow_Statement.columns[1]] = Cash_Flow_Statement[Cash_Flow_Statement.columns[1]].map(lambda x: x.replace('nan','0'))
            Cash_Flow_Statement[Cash_Flow_Statement.columns[2]] = Cash_Flow_Statement[Cash_Flow_Statement.columns[2]].map(lambda x: x.replace('nan','0'))
            
            
            Cash_Flow_Statement[Cash_Flow_Statement.columns[1:]] = Cash_Flow_Statement[Cash_Flow_Statement.columns[1:]].astype(float)
            
            cash_flow_statements.append(Cash_Flow_Statement)
        

In [7]:
#This will clean up the SEC tsv files after running the script and extracting the relevant information.
def purge_tsv_files():
    for file in tsv_files:
        os.remove(directory_path + file)
        print('File {} deleted'.format(file))

# Program

In [8]:
#Downloads the list of all company files including the location of those files for us to look through.
if os.path.exists(directory_path + str(year) + '-' + 'QTR1.tsv') == True:
    tsv_name_formatter()
    tsv_files = os.listdir(directory_path)
else:
    edgar.download_index(directory_path, year,
                         skip_all_present_except_last=False)
    print('Files downloaded successfully')
    tsv_files = os.listdir(directory_path)
    tsv_name_formatter()


Files downloaded successfully
2020-QTR1.tsv was created
2020-QTR2.tsv was created
2020-QTR3.tsv was created


In [12]:
#This for loop will take all the tsv files that were downloaded for the year and compile the different 
#financial reports
for file in tsv_file_names:
    csv = pd.read_csv('/home/user/Documents/python_webscraper/' + file + '.tsv', 
                  sep='\t',  lineterminator='\n', names=None) 

    csv.columns.values[0] = 'Item'
    
    #This for loop ensures that the proper documents are found regardless of whether we are looking at a 10-Q or 10-K
    if file is not (str(year) + '-QTR4'):
        selectedreport = '10-Q'
    else:
        selectedreport = '10-K'
        
    companyreport = csv[(csv['Item'].str.contains(selectedcompany)) & 
                        (csv['Item'].str.contains(selectedreport))]

    Filing = companyreport['Item'].str.split('|')
    Filing = Filing.to_list()

    for item in Filing[0]:

        if 'html' in item:
            report = item

    url = 'https://www.sec.gov/Archives/' + report
    #https://www.sec.gov/ix?doc=/Archives/edgar/data/1652044/000165204419000032/goog10-qq32019.htm

    df = pd.read_html(url)
    document_index = df[0]
    document_index = document_index.dropna()

    document_name = document_index[document_index['Description'].str.contains(selectedreport)]
    document_name = document_name['Document'].str.split(' ')
    document_name = document_name[0][0]

    report_formatted = report.replace('-','').replace('index.html','')
    url = 'https://www.sec.gov/Archives/' + report_formatted + '/' + document_name


    df = pd.read_html(url)
    
    #invoking the balance_sheet definition to compile the report
    balance_sheet()
    
    cash_flow_statment()

ValueError: Columns must be same length as key

This section is where the tsv files will be used to compile the different reports. Below is the Balance Sheet as the first one.

In [10]:
#This is used at the end of the script to clean up any residual junk files that will not be needed. It will delete 
#all the tsv files that were downloaded at the beginning. 
if purge_file_directory is True:
    purge_tsv_files()


File 2020-QTR1.tsv deleted
File 2020-QTR2.tsv deleted
File 2020-QTR3.tsv deleted
